Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Радует, что ноутбук хорошо структурирован. Приятно проверять такие работы.
* Отлично, что стоп-слова были исключены при векторизации!
* Работа получилась отличной, тебе удалось добиться достаточно хорошего качества. Поздравляю!
* Проект зачтен, удачи в дальнейшем обучении!

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTree" data-toc-modified-id="DecisionTree-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>DecisionTree</a></span></li><li><span><a href="#RandomForest" data-toc-modified-id="RandomForest-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>RandomForest</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.stem import WordNetLemmatizer 
import re 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
df = pd.read_csv("https://code.s3.yandex.net/datasets/toxic_comments.csv")


Проведем первичный анализ датасета.

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Вывод: с данными все в норме, можно работать дальше.

<div class="alert alert-block alert-success">
<b>Успех:</b> Загрузка и первичный осмотр данных проведены хорошо.
</div>

In [6]:
X = df["text"].values.astype('U')

In [7]:
y = df["toxic"]

In [8]:
#функция для лемматизации
def lemma(text):
    l = WordNetLemmatizer()
    return ["".join(l.lemmatize(w)) for w in text]

<div class="alert alert-block alert-info">
<b>Совет: </b> Таким образом ты применяешь лемматизатор к буквам, что не имеет смысла.
</div>

In [9]:
#функция для очистки комментариев от ненужных символов
def clear_text(text):
    return [" ".join((re.sub('[^a-zA-Z]', ' ', w)).split()) for w in text]

In [10]:
X = lemma(clear_text(X))

<div class="alert alert-block alert-success">
<b>Успех:</b> Радует, что сначала была проведена очистка, а затем лемматизация.
</div>

In [11]:
X_train, X_test, y_train, y_test =\
train_test_split(X, y, test_size=0.2, random_state=47)

In [12]:
stopwords = set(nltk_stopwords.words('russian'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords, min_df = 0.0001)

In [13]:
count_tf_idf.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=0.0001, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'а', 'без', 'более', 'больше', 'будет', 'будто',
                            'бы', 'был', 'была', 'были', 'было', 'быть', 'в',
                            'вам', 'вас', 'вдруг', 'ведь', 'во', 'вот',
                            'впрочем', 'все', 'всегда', 'всего', 'всех', 'всю',
                            'вы', 'где', 'да', 'даже', 'два', ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [14]:
X_train = count_tf_idf.transform(X_train)

In [15]:
X_test = count_tf_idf.transform(X_test)

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что обучил векторизатор только на тренировочной части данных. Это уменьшает переобучение.
</div>

In [16]:
#функция, которая применяет методы fit, predict и считает и выводит F score
def fit_predict_evaluate(model, X_train=X_train, y_train=y_train, 
                         y_test=y_test, X_test=X_test):
    model.fit(X_train, y_train)
    return print("F score: ", f1_score(y_test, model.predict(X_test)))

In [17]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(127656, 16500) (31915, 16500) (127656,) (31915,)


## Обучение

Используем написанную нами функцию, для ыстрой оценки моделей.

### LogisticRegression

In [18]:
model = LogisticRegression()

In [19]:
fit_predict_evaluate(model)

F score:  0.7521897810218978


### DecisionTree

In [20]:
model = DecisionTreeClassifier()

In [21]:
fit_predict_evaluate(model)

F score:  0.6868623450007848


### RandomForest

In [22]:
model = RandomForestClassifier()

In [23]:
fit_predict_evaluate(model)

F score:  0.6477683956574186


Вывод: по результатам нашего теста, лучше всего работает логистическая регрессия. Её результат 0,752. Попробуем его улучшить с помощью перебора параметров по сетке.

In [24]:
grid={"C":np.logspace(-3,3,3), "penalty":["l1","l2"], 
      "class_weight": ['balanced', None]}

In [25]:
model = LogisticRegression()

In [ ]:
clf = GridSearchCV(estimator=model, param_grid=grid,  cv=3, scoring="f1")
fit_predict_evaluate(clf)

Подбор параметров улучшил результат до 0.7667.

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что попробовал несколько разных моделей и для подбора параметров использовал метод основанный на кросс-валидации с корректной метрикой.
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Можно было еще попробовать catboost, у него есть внутренний механизм для работы с текстами.
</div>

Попробуем изменить количество итераций.

In [ ]:
model = LogisticRegression(C=1000.0, class_weight=None, penalty= 'l2', 
                           max_iter=50)

In [ ]:
fit_predict_evaluate(model)

In [ ]:
precision_score(y_test, model.predict(X_test))

In [ ]:
recall_score(y_test, model.predict(X_test))


## Выводы

Исходя из проделанной работы, мы можем сделать вывод, что луший результат дала модель LogisticRegression с параметрами C=1000.0, class_weight=None, penalty= 'l2', max_iter=50. F score получился равным 0.7972.

<div class="alert alert-block alert-success">
<b>Успех:</b> Тебе удалось добиться очень хорошего качества, молодец!
</div>

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны